In [16]:
import torch.nn as nn
import gpytorch
from sklearn.datasets import load_boston

In [7]:
data = load_boston()

In [10]:
X = data['data']
y = data['target']

In [11]:
import gpytorch

In [15]:
gpytorch.models.ExactGP

gpytorch.models.exact_gp.ExactGP

In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
likelihood.train()


In [ ]:
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


In [ ]:
def train():
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()

In [ ]:
class GP_prior(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kld_scale):
        super(GP_prior, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.cov_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                ard_num_dims = train_x.shape[-1]
            )
        )
        self.cov_module.initialize_from_data(train_x, train_y)

    def forward(self, x):
        return gpytorch.distributions.MultivariateNormal(
            self.mean_module(x),
            self.cov_module(x),
        )